In [1]:
import os
os.getcwd()

'c:\\Users\\mark7\\MLDS\\return_forecasting\\data_engineering'

In [2]:
%run "import_raw.ipynb"

In [3]:
df_raw.shape

(21864405, 16)

In [4]:
df = df_raw.copy()
del df_raw

In [5]:
df.shape

(21864405, 16)

### Note:
This is what *Mark* do on the raw dataset, his priciple is to remove null data and extreme records. Because it's nearly impossible for such a thing to happen.

### Basic Principle:
It makes no sense that a product's selling price is way higher or way lower than the original price. And if that do happened in the real life, we don't want to learn from these extreme records because they misguide our model.

### An example:
remove records that (AMT * 0.1) > ORGPRICE, sell too high to make it unbelievable, either outlier or typo.

### 1. orgprice vs amt

In [6]:
df_sell_high = df[df['AMT'] > df['ORGPRICE']]

In [7]:
df_todrop_1 = df_sell_high[df_sell_high['AMT'] * 0.1 > df_sell_high['ORGPRICE']]
print(df_todrop_1[['AMT', 'ORGPRICE']].head())
len(df_todrop_1)

            AMT  ORGPRICE
575816    22.00      1.99
1536600   60.00      5.00
3061404  189.99     12.00
3061406  189.99     12.00
9289303  120.00      7.00


31

In [8]:
df_sell_high_droped = df_sell_high[df_sell_high['AMT'] * 0.1 <= df_sell_high['ORGPRICE']]
len(df_sell_high_droped)
# 9147 -> 9116

9116

In [9]:
# discount (sell low)
df_sell_low = df[df['AMT'] < df['ORGPRICE']]
df_sell_low.head()

,SKU,SALEDATE,INTERID,STYPE,QUANTITY,ORGPRICE,AMT,CITY,STATE,COST,RETAIL,DEPT,COLOR,SIZE,BRAND,ZIP
7,1000057,2005-08-21,391902128,P,1,100.0,50.00,MCALLEN,TX,41.65,50.00,3701.0,AMET,22,POSITIVE,78501
8,1000057,2005-08-22,929307832,R,1,100.0,67.00,SAN ANTONIO,TX,41.65,50.00,3701.0,AMET,22,POSITIVE,78216
9,1000057,2005-08-25,748505965,P,1,100.0,41.99,HAMMOND,LA,37.13,59.99,3701.0,AMET,22,POSITIVE,70403
19,1000129,2005-02-17,839102215,P,1,35.0,8.75,WICHITA,KS,16.00,8.75,8306.0,NAVY,120M,KENNETH,67207
20,1000129,2005-02-23,572006110,P,1,35.0,4.37,WICHITA,KS,16.00,8.75,8306.0,NAVY,120M,KENNETH,67209


In [10]:
# sell too low
df_todrop_2 = df_sell_low[df_sell_low['AMT'] < df_sell_low['ORGPRICE'] * 0.1]
print(df_todrop_2[['AMT', 'ORGPRICE']].head())
len(df_todrop_2)

      AMT  ORGPRICE
478  3.49      38.0
480  3.49      38.0
503  3.49      38.0
505  3.49      38.0
509  3.49      38.0


46802

In [11]:
# drop sell too low
df_sell_low_droped = df_sell_low[df_sell_low['AMT'] >= df_sell_low['ORGPRICE'] * 0.1]
len(df_sell_low_droped)
# 10009465 -> 9962663  

9962663

In [12]:
# equal
df_sell_fair = df[df['AMT'] == df['ORGPRICE']]
len(df_sell_fair)

11845793

In [13]:
df_sell_fair.shape

(11845793, 16)

In [14]:
df_sell_low_droped.shape

(9962663, 16)

In [15]:
df_sell_high_droped.shape

(9116, 16)

### 2. cost vs retail

In [16]:
df_todrop_1plus = df_sell_high_droped[df_sell_high_droped['RETAIL'] * 0.1 > df_sell_high_droped['COST']]
df_todrop_1plus[['RETAIL','COST']].head()

,RETAIL,COST
2819198,14.63,0.0
7826302,14.63,0.0
10204060,2.49,0.0


In [17]:
df_todrop_2plus = df_sell_low_droped[df_sell_low_droped['RETAIL'] < df_sell_low_droped['COST'] * 0.1]
df_todrop_2plus[['RETAIL','COST']].head()

,RETAIL,COST
52578,2.99,36.12
58389,1.00,16.50
71077,5.99,72.50
86104,1.00,14.25
86111,1.00,14.25


In [18]:
df_sell_high_droped_2 = df_sell_high_droped[df_sell_high_droped['RETAIL'] * 0.1 <= df_sell_high_droped['COST']]
df_sell_high_droped_2.shape
# 9116 -> 9113

(9113, 16)

In [19]:
df_sell_low_droped_2 = df_sell_low_droped[df_sell_low_droped['RETAIL'] >= df_sell_low_droped['COST'] * 0.1]
df_sell_low_droped_2.shape
# 9962663 -> 9962663

(9961113, 16)

In [20]:
## After Cleaning
df_removed = pd.concat([df_sell_fair, df_sell_high_droped_2, df_sell_low_droped_2], ignore_index=True)
df_removed.shape

(21816019, 16)